In [7]:
import io
import struct
from configs import constants

class Reader:
    reader: io.BufferedReader
    data: bytes
    buffer_value : any 

    def __init__(self, reader: io.BufferedReader = None):
        if reader is None:
            raise ValueError("Reader: None reader given")
        self.reader = reader
        self.data = b""

    def read(self,size:int = None):
        if self.reader is None:
            raise ValueError("Plz init the reader jeezzz")
        
        self.data = self.reader.read(size)
        return self.data 

    def read_int(self):
        return int.from_bytes(self.read(size = constants.LenInt32 ))
    
    def read_byte(self):
        return self.read(size = constants.LenInt32 )

    def read_tlv(self):
        buffer = io.BytesIO()
        datatype = self.read_byte()
        buffer.write(datatype)

        length = self.read_int()
        buffer.write(length.to_bytes(constants.LenInt32))

        value = self.read(length)
        buffer.write(value)
        self.buffer_value =  buffer.getvalue()
        return self.buffer_value




In [8]:
stream = io.BytesIO(b'\x01\x00\x00\x00\xff\xff\xff\xff')

In [2]:
stream = io.BytesIO(b'\x01\x00\x00\x00\xff\xff\xff\xffHello!')

r = Reader(stream)



In [3]:
r.read()

b'\x01\x00\x00\x00\xff\xff\xff\xffHello!'

In [9]:
from marshaller.valmarshal import TLVMarshaler
tlv = TLVMarshaler(value = "Shreyank")

In [10]:
tlv.tlv_marshal()

Encoded TLV: [2, 8, 0, 0, 0, 83, 104, 114, 101, 121, 97, 110, 107] for value Shreyank


b'\x02\x08\x00\x00\x00Shreyank'

In [11]:
stream = io.BytesIO(tlv.tlv)
r= Reader(stream)
r.read_tlv()

b'\x02\x08\x00\x00\x00Shreyank'

In [12]:
binary_data = b'\x02\x04\x00\x00\x00demo'
stream = io.BytesIO(binary_data)
r = Reader(stream)
r.read_tlv()

b'\x02\x04\x00\x00\x00demo'

In [1]:
from modals.db import DataBase
import modals

db = DataBase("t1db")
columns_map = {"id": modals.columns.Column("id", 1, False)}  # Assume
columns_list = ["id"]
table = db.create_tables("mytable", columns_list, columns_map)

Encoded TLV: [2, 2, 0, 0, 0, 105, 100] for value id
Encoded TLV: [5, 4, 0, 0, 0, 1, 0, 0, 0] for value 1
Encoded TLV: [4, 1, 0, 0, 0, 0] for value False
The value b'c\x00\x00\x00\x12\x00\x00\x00\x02\x02\x00\x00\x00id\x05\x04\x00\x00\x00\x01\x00\x00\x00\x04\x01\x00\x00\x00\x00' was wrote to the colwriter of length 30 bytes with total data being 30


In [4]:
t = db.tables['mytable']

In [6]:
c_obj = t.columns[t.columnNames[-1]]

In [9]:
m = c_obj.column_marshaler

In [11]:
m.print_bytes()

the byte value is [99, 0, 0, 0, 18, 0, 0, 0, 2, 2, 0, 0, 0, 105, 100, 5, 4, 0, 0, 0, 1, 0, 0, 0, 4, 1, 0, 0, 0, 0]


In [12]:
from marshaller.colmarshal import ColumnUnmarshaler
un = ColumnUnmarshaler(m.column_tlv_encoded)

In [14]:
m.column_tlv_encoded

b'c\x00\x00\x00\x12\x00\x00\x00\x02\x02\x00\x00\x00id\x05\x04\x00\x00\x00\x01\x00\x00\x00\x04\x01\x00\x00\x00\x00'

In [15]:
import marshaller
import marshaller.colmarshal

m = marshaller.colmarshal.ColumnMarshaler(name = "id",datatype=1,allowNull=False)

In [16]:
m.marshal_column()

Encoded TLV: [2, 2, 0, 0, 0, 105, 100] for value id
Encoded TLV: [5, 4, 0, 0, 0, 1, 0, 0, 0] for value 1
Encoded TLV: [4, 1, 0, 0, 0, 0] for value False


b'c\x00\x00\x00\x12\x00\x00\x00\x02\x02\x00\x00\x00id\x05\x04\x00\x00\x00\x01\x00\x00\x00\x04\x01\x00\x00\x00\x00'

In [18]:
m.print_bytes()

the byte value is [99, 0, 0, 0, 18, 0, 0, 0, 2, 2, 0, 0, 0, 105, 100, 5, 4, 0, 0, 0, 1, 0, 0, 0, 4, 1, 0, 0, 0, 0]


In [19]:
u = marshaller.colmarshal.ColumnUnmarshaler(m.column_tlv_encoded)

In [35]:
u = ColumnUnmarshaler(m.column_tlv_encoded)
c = u.unmarshal_column()

Outer type is  99
Column length is  18
Decoded TLV: type=2, length=2, value=id
Name field decoded at offset 15
Decoded TLV: type=5, length=4, value=1
Datatype field decoded at offset 24
Decoded TLV: type=4, length=1, value=False
AllowNull field decoded at offset 30


In [37]:
c.name,c.allow_null,c.datatype

('id', False, 1)

In [1]:
import modals.columns
from modals.db import DataBase
import modals

db = DataBase("t3db")
columns_map = {"id": modals.columns.Column("id", 1, False),"name":modals.columns.Column("name",2,True)}  
columns_list = ["id","name"]
table = db.create_tables("mytable", columns_list, columns_map)

Encoded TLV: [2, 2, 0, 0, 0, 105, 100] for value id
Encoded TLV: [5, 4, 0, 0, 0, 1, 0, 0, 0] for value 1
Encoded TLV: [4, 1, 0, 0, 0, 0] for value False
The value b'c\x00\x00\x00\x12\x00\x00\x00\x02\x02\x00\x00\x00id\x05\x04\x00\x00\x00\x01\x00\x00\x00\x04\x01\x00\x00\x00\x00' was wrote to the colwriter of length 30 bytes with total data being 30
Encoded TLV: [2, 4, 0, 0, 0, 110, 97, 109, 101] for value name
Encoded TLV: [5, 4, 0, 0, 0, 2, 0, 0, 0] for value 2
Encoded TLV: [4, 1, 0, 0, 0, 1] for value True
The value b'c\x00\x00\x00\x13\x00\x00\x00\x02\x04\x00\x00\x00name\x05\x04\x00\x00\x00\x02\x00\x00\x00\x04\x01\x00\x00\x00\x01' was wrote to the colwriter of length 32 bytes with total data being 32


In [2]:
id = table.columns['id']

In [3]:
name = table.columns['name']

In [4]:
mid = id.column_marshaler.marshal_column()

Encoded TLV: [2, 2, 0, 0, 0, 105, 100] for value id
Encoded TLV: [5, 4, 0, 0, 0, 1, 0, 0, 0] for value 1
Encoded TLV: [4, 1, 0, 0, 0, 0] for value False


In [5]:
type(mid)

bytes

In [6]:
import marshaller
import marshaller.colmarshal
u = marshaller.colmarshal.ColumnUnmarshaler(mid)

In [7]:
u.unmarshal_column()
u.print_col()

Outer type is  99
Column length is  18
Decoded TLV: type=2, length=2, value=id
Name field decoded at offset 15
Decoded TLV: type=5, length=4, value=1
Datatype field decoded at offset 24
Decoded TLV: type=4, length=1, value=False
AllowNull field decoded at offset 30
the column name is id with the dtype 1 and has allownull False


In [8]:
name = name.column_marshaler.marshal_column()

Encoded TLV: [2, 4, 0, 0, 0, 110, 97, 109, 101] for value name
Encoded TLV: [5, 4, 0, 0, 0, 2, 0, 0, 0] for value 2
Encoded TLV: [4, 1, 0, 0, 0, 1] for value True


In [10]:
u = marshaller.colmarshal.ColumnUnmarshaler(name)

In [11]:
u.unmarshal_column()

Outer type is  99
Column length is  19
Decoded TLV: type=2, length=4, value=name
Name field decoded at offset 17
Decoded TLV: type=5, length=4, value=2
Datatype field decoded at offset 26
Decoded TLV: type=4, length=1, value=True
AllowNull field decoded at offset 32


In [12]:
u.print_col()

the column name is name with the dtype 2 and has allownull True
